In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import torch
import os

from PIL import Image

import os
import requests
from PIL import Image
from io import BytesIO
import torchvision.transforms as T
from tqdm import tqdm

transform = T.ToPILImage()
import torch.optim as optim
import json
import numpy as np


In [3]:
from model.openllama import OpenLLAMAPEFTModel


/home/eugene/miniconda3/envs/imagebind/lib/python3.8/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/home/eugene/miniconda3/envs/imagebind/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/eugene/miniconda3/envs/imagebind/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' m

In [ ]:
import data
import torch
from model.ImageBind.models import imagebind_model
from model.ImageBind.models.imagebind_model import ModalityType

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate model
imagebind = imagebind_model.imagebind_huge(pretrained=True)
imagebind = imagebind[0]
imagebind.eval()
imagebind.to(device)
0

In [4]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        tensor = tensor.clone()
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

DEFAULT_IMAGE_TOKEN = "<image>"
DEFAULT_IMAGE_PATCH_TOKEN = "<im_patch>"
DEFAULT_IM_START_TOKEN = "<im_start>"
DEFAULT_IM_END_TOKEN = "<im_end>"

In [5]:
args = {
    'model': 'openllama_peft',
    'imagebind_ckpt_path': './models/converted/imagebind/',
    'vicuna_ckpt_path': './models/converted/vicuna_full/',
    'delta_ckpt_path': './models/converted/pandagpt_7b_max_len_1024/pytorch_model.pt',
    'stage': 2,
    'max_tgt_len': 128,
    'lora_r': 32,
    'lora_alpha': 32,
    'lora_dropout': 0.1,
}

**We load ImageBind on GPU and PandaGPT on CPU** Please decide based
on your GPU size

In [6]:
model = OpenLLAMAPEFTModel(**args)
delta_ckpt = torch.load(args['delta_ckpt_path'], map_location=torch.device('cpu'))
model.load_state_dict(delta_ckpt, strict=False)


Initializing visual encoder from /home/eugene/models/converted/imagebind/ ...
Visual encoder initialized.
Initializing language decoder from /home/eugene/models/converted/vicuna_full/ ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 33554432 || all params: 6771978240 || trainable%: 0.49548936530546206
Language decoder initialized.


_IncompatibleKeys(missing_keys=['visual_encoder.modality_preprocessors.vision.cls_token', 'visual_encoder.modality_preprocessors.vision.rgbt_stem.proj.1.weight', 'visual_encoder.modality_preprocessors.vision.pos_embedding_helper.pos_embed', 'visual_encoder.modality_preprocessors.text.pos_embed', 'visual_encoder.modality_preprocessors.text.mask', 'visual_encoder.modality_preprocessors.text.token_embedding.weight', 'visual_encoder.modality_preprocessors.audio.cls_token', 'visual_encoder.modality_preprocessors.audio.rgbt_stem.proj.weight', 'visual_encoder.modality_preprocessors.audio.rgbt_stem.norm_layer.weight', 'visual_encoder.modality_preprocessors.audio.rgbt_stem.norm_layer.bias', 'visual_encoder.modality_preprocessors.audio.pos_embedding_helper.pos_embed', 'visual_encoder.modality_preprocessors.depth.cls_token', 'visual_encoder.modality_preprocessors.depth.depth_stem.proj.weight', 'visual_encoder.modality_preprocessors.depth.depth_stem.norm_layer.weight', 'visual_encoder.modality_pre

In [236]:
def get_id(path, audio=True):
    modality_cache=[]
    if audio:
        prompt_text = 'What is this sound?'
        audio_path = path
        image_path = None
    else:
        prompt_text = 'What is on this image?'
        audio_path = None
        image_path = path
    
    response = model.generate({
            'prompt': prompt_text,
            'image_paths': [image_path] if image_path else [],
            'audio_paths': [audio_path] if audio_path else [],
            'video_paths': [video_path] if video_path else [],
            'thermal_paths': [thermal_path] if thermal_path else [],
            'top_p': top_p,
            'temperature': temperature,
            'max_tgt_len': max_length,
            'modality_embeds': modality_cache
        })
    return response

In [217]:
temperature=0.2

In [131]:
get_id(path='/home/eugene/ImageBind/hack/audio_wolves_to_sheep.pt', audio=True)

IMPORTANT!! OVERWRITING AUDIO LOADING WITH MODIFIED INPUT: /home/eugene/ImageBind/hack/audio_wolves_to_sheep.pt
feature_embeds:  torch.Size([1, 1, 4096])
p_after_embeds torch.Size([1, 15, 4096])
inputs_embeds torch.Size([1, 23, 4096])
input_embeds torch.Size([1, 23, 4096])


'The sound of a sheep bleating can be described as a soft, low-pitched, and continuous noise. It is a common'

In [240]:
get_id(path='/home/eugene/ImageBind/hack/audio_wolves_to_sheep.pt', audio=True)

IMPORTANT!! OVERWRITING AUDIO LOADING WITH MODIFIED INPUT: /home/eugene/ImageBind/hack/audio_wolves_to_sheep.pt
feature_embeds:  torch.Size([1, 1, 4096])
p_after_embeds torch.Size([1, 14, 4096])
inputs_embeds torch.Size([1, 22, 4096])
input_embeds torch.Size([1, 22, 4096])


'The sound in the image is a sheep bleating, which is a common vocalization made by sheep. The sheep is standing on a grassy hillside, and its bleating can be heard in the background. This sound is a natural part of the communication and social behavior of sheep, and it can indicate various emotions or needs, such as seeking attention, expressing discomfort, or calling for companionship among other sheep.'

In [238]:
get_id(path='/home/eugene/ImageBind/all_assets/chains.wav', audio=True)

feature_embeds:  torch.Size([1, 1, 4096])
p_after_embeds torch.Size([1, 14, 4096])
inputs_embeds torch.Size([1, 22, 4096])
input_embeds torch.Size([1, 22, 4096])


'The sound in the image is a rattling noise coming from a bicycle wheel. The wheel is attached to the bike and appears to be loose or not properly secured, causing the rattling noise. This could be due to a worn-out or damaged wheel, a loose spoke, or an improperly tightened wheel nut. The rattling noise might be concerning for the bicycle owner, as it could indicate a potential safety issue or a need for maintenance to prevent further damage to the wheel or the bike itself.'

In [239]:
get_id(path='/home/eugene/ImageBind/all_assets/wolves.wav', audio=True)

feature_embeds:  torch.Size([1, 1, 4096])
p_after_embeds torch.Size([1, 14, 4096])
inputs_embeds torch.Size([1, 22, 4096])
input_embeds torch.Size([1, 22, 4096])


'The sound is a dog barking.'

In [237]:
get_id(path='/home/eugene/ImageBind/all_assets/wolves.wav', audio=True)

feature_embeds:  torch.Size([1, 1, 4096])
p_after_embeds torch.Size([1, 14, 4096])
inputs_embeds torch.Size([1, 22, 4096])
input_embeds torch.Size([1, 22, 4096])


'The sound is a dog barking.'

In [230]:
get_id(path='/home/eugene/ImageBind/all_assets/wolves.wav', audio=True)

feature_embeds:  torch.Size([1, 1, 4096])
p_after_embeds torch.Size([1, 15, 4096])
inputs_embeds torch.Size([1, 23, 4096])
input_embeds torch.Size([1, 23, 4096])


"The image shows a white wolf standing on its hind legs with its front paws resting on its chest. The wolf's ears are perked up and its eyes are focused on something in the distance. The wolf's mouth is slightly open, and its tongue is hanging out. The wolf appears to be alert and attentive, possibly reacting to a sound or smell in the environment."